In [252]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [253]:
con = pd.read_csv('../data/전처리data/click.csv', encoding='cp949')
con.columns = ['subsr', 'program', 'vod_id', 'main_cat', 'sub_cat', 'category', 'click_score']
con

,subsr,program,vod_id,main_cat,sub_cat,category,click_score
0,65968000,핑크퐁 동물 동요,882,키즈,기타,키즈,1.000000
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.012195
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.006098
3,62733000,엘리가간다 시즌12,1958,키즈,기타,키즈,0.500000
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.012195
...,...,...,...,...,...,...,...
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455
7354,59917000,라바 인 뉴욕,10,키즈,오락,키즈,0.500000
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098


In [254]:
vod_list = pd.read_csv('../data/전처리data/vod_list_add10.csv', index_col=0)
vod_id = vod_list[['rename', 'Category', 'vod_id']]
vod_id.columns = ['rename', 'category', 'vod_id']
vod_id

,rename,category,vod_id
0,와이낫크루뜻밖의여행,TV프로그램,0
1,그레이트뷰티,영화,1
2,해안선,영화,2
3,아는형님,TV프로그램,3
4,베놈,영화,4
...,...,...,...
5019,일루셔니스트,영화,5019
5020,"파리,13구",영화,5020
5021,미쓰백,영화,5021
5022,그녀는거짓말을너무사랑해,영화,5022


In [255]:
con['subsr'].value_counts()

60939000    446
66705000    230
64984000    189
64124000    162
64659000    152
           ... 
63748000      1
66111000      1
62017000      1
65358000      1
65968000      1
Name: subsr, Length: 899, dtype: int64

In [270]:
# con1 = con.copy()

In [257]:
# 3개 미만 삭제 (총 유저 899명, 시청 기록이 1 or 2인 user 454명)
value_counts = con['subsr'].value_counts()
to_remove = value_counts[value_counts < 3].index
con1 = con[~con['subsr'].isin(to_remove)]
con1

,subsr,program,vod_id,main_cat,sub_cat,category,click_score
1,63086000,지니강이 플러스 시즌7,1622,키즈,기타,키즈,0.012195
2,63086000,지니강이 플러스 시즌9,2043,키즈,기타,키즈,0.006098
4,63086000,지니강이 플러스 시즌8,1596,키즈,기타,키즈,0.012195
5,63086000,헤이지니 럭키강이 Part1,4036,키즈,기타,키즈,0.006098
6,63086000,우당탕탕 에디의 실험실,1834,키즈,기타,키즈,0.012195
...,...,...,...,...,...,...,...
7351,60040000,포켓몬스터: 리코와 로드의 모험,1273,키즈,애니메이션,키즈,0.076923
7352,64982000,포켓몬스터 XY and Z,2918,키즈,애니메이션,키즈,0.014493
7353,66864000,포켓몬스터 썬 앤 문 3,3135,키즈,애니메이션,키즈,0.045455
7355,63086000,사이틴 시즌1,3468,키즈,학습,키즈,0.006098


In [271]:
con_score = con1[['subsr', 'vod_id', 'click_score']]
con_score

,subsr,vod_id,click_score
1,63086000,1622,0.012195
2,63086000,2043,0.006098
4,63086000,1596,0.012195
5,63086000,4036,0.006098
6,63086000,1834,0.012195
...,...,...,...
7351,60040000,1273,0.076923
7352,64982000,2918,0.014493
7353,66864000,3135,0.045455
7355,63086000,3468,0.006098


In [78]:
! pip install surprise

In [273]:
# 필요한 Surprise 알고리즘 불러오기
from surprise import SVD, BaselineOnly, SVDpp, Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

# surprise 데이터 형식으로 변환
def convert_traintest_dataframe_forsurprise(training_dataframe):
    reader = Reader(rating_scale=(0, 1)) # 범위가 0~1 인 경우
    trainset = Dataset.load_from_df(training_dataframe[['subsr', 'vod_id', 'click_score']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    return trainset

trainset = convert_traintest_dataframe_forsurprise(con_score)

### KNNBaseline - user-based similarity
- cosine (0.026153846153846153)
- pearson_baseline (0.025641025641025633)
  - k = 10 (0.025641025641025633)
  - k = 50 (0.025641025641025633)
  - k = 80 (0.025641025641025633)
  - k = 80, n_factors = 100 (0.025641025641025633)
- msd (0.018974358974358986)
- **pearson, user_based: True , k = 30 (0.03282051282051279)**
  - n_factors = 100 추가 (0.03282051282051279)
  - k = 80, n_factors = 100 (0.03282051282051279)

(vod에 있는 사용자 제거하고 사용)
#### 3개 미만
- cosine (0.00625)
- msd, k = 30 (0.00625)
- pearson (0.00625)
- pearson_baseline (0.0)

#### 전체
- pearson_baseline, k=100 (0.010256410256410256)
- cosine, k = 100 (0.007692307692307694)
  - n_factors = 100 (0.007692307692307694)
- msd, k = 100, n_factors = 100 (0.007692307692307694)
- pearson (0.015384615384615384)

In [289]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'k': (1, 100),
    'sim_options': {'name': ['pearson'], 'n_factors': [100, 200, 300]}
}

gs = GridSearchCV(KNNBaseline, param_grid, measures=['mae'], cv=5)

# 데이터에 대해 그리드 탐색 수행
reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als.

{'mae': {'k': 100,
  'sim_options': {'name': 'pearson', 'n_factors': 100, 'user_based': True}}}

In [246]:
sim_options = {'name': 'pearson',
'user_based': True}

bsl_options = {'method' : 'sgd', 'n_epochs' : 1}
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline_u = KNNBaseline(k = 30, sim_options=sim_options, random_state = 42)
knnbaseline_u.fit(trainset)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [247]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline_u.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.089332
1,59879000,1,None,0.091833
2,59879000,3,None,0.089181
3,59879000,5,None,0.089213
4,59879000,6,None,0.115409
...,...,...,...,...
1174350,67160000,4708,None,0.067741
1174351,67160000,4710,None,0.069055
1174352,67160000,4711,None,0.068609
1174353,67160000,4712,None,0.068272


In [ ]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.089332
1,59879000,1,0.091833
2,59879000,3,0.089181
3,59879000,5,0.089213
4,59879000,6,0.088572
...,...,...,...
1174350,67160000,4708,0.057524
1174351,67160000,4710,0.071981
1174352,67160000,4711,0.068609
1174353,67160000,4712,0.063365


In [248]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2092,143,1166,2537,296,8,2288,2865,1329,...,1200,652,1243,7,256,553,2336,518,780,432
1,59882000,1233,3026,143,1267,296,336,2057,1329,297,...,4167,2288,3462,553,2336,2321,780,432,2624,123
2,59886000,3891,818,1398,143,1166,2537,296,8,2288,...,335,784,1200,652,1243,7,256,553,2336,518
3,59892000,143,8,2288,2865,1329,1267,296,240,2537,...,3838,335,784,1200,652,1243,7,256,553,2336
4,59895000,784,249,143,2537,1166,8,2633,2288,432,...,335,1200,1243,7,256,553,518,780,123,2058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,67130000,143,476,1166,296,8,888,1329,738,3576,...,784,1243,2172,1629,256,553,2336,648,780,432
441,67137000,2057,2866,910,143,1166,2537,296,8,2288,...,784,1200,652,1243,7,256,553,2336,518,780
442,67140000,1166,738,748,784,2560,2057,610,1191,2865,...,297,2045,225,245,554,3838,335,4138,531,2336
443,67149000,297,2226,610,991,143,726,2057,8,652,...,3838,335,116,784,1200,1243,553,518,2537,780


In [240]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[1233, 728, 296, 2092, 8, 759, 2865, 1329, 115..."
1,59882000,"[335, 1401, 1394, 754, 2226, 143, 610, 336, 55..."
2,59886000,"[738, 3891, 3954, 610, 448, 36, 8, 757, 16, 12..."
3,59892000,"[1243, 139, 3779, 1544, 754, 3838, 518, 3307, ..."
4,59895000,"[518, 3779, 3838, 3307, 1242, 2226, 1150, 738,..."
...,...,...
440,67130000,"[335, 610, 249, 336, 552, 709, 4052, 862, 2879..."
441,67137000,"[241, 315, 139, 2592, 245, 2226, 738, 2057, 29..."
442,67140000,"[498, 139, 2148, 1243, 2592, 3779, 2986, 245, ..."
443,67149000,"[3838, 315, 498, 738, 2592, 2986, 245, 754, 33..."


In [241]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[1233.0, 728.0, 296.0, 2092.0, 8.0, 759.0, 286..."
1,59882000,"[335.0, 1401.0, 1394.0, 754.0, 2226.0, 143.0, ..."
2,59886000,"[738.0, 3891.0, 3954.0, 610.0, 448.0, 36.0, 8...."
3,59892000,"[1243.0, 139.0, 3779.0, 1544.0, 754.0, 3838.0,..."
4,59895000,"[518.0, 3779.0, 3838.0, 3307.0, 1242.0, 2226.0..."
...,...,...
440,67130000,"[335.0, 610.0, 249.0, 336.0, 552.0, 709.0, 405..."
441,67137000,"[241.0, 315.0, 139.0, 2592.0, 245.0, 2226.0, 7..."
442,67140000,"[498.0, 139.0, 2148.0, 1243.0, 2592.0, 3779.0,..."
443,67149000,"[3838.0, 315.0, 498.0, 738.0, 2592.0, 2986.0, ..."


In [ ]:
testdata = pd.read_csv('../data/전처리data/novod_user.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

In [234]:
# test_data와 예측 값의 subsr 비교
en_predict = vod_predict['subsr'].to_list()
t_data = testdata['subsr'].to_list()
t_en = set(t_data) - set(en_predict)
len(t_en)

36

In [242]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.004545454545454546

### KNNBaseline - item-based similarity
- cosine (0.013333333333333338)
- **pearson_baseline, k = 6 (0.025128205128205124)**
  - k = 80, n_factors = 100 (0.02256410256410257)
- msd (0.011282051282051288)

#### 3개 미만
- pearson, k = 50 (0.00625)
- cosine, k = 50 (0.0)
- msd (0.0)
- pearson_baseline(0.00625)

#### 전체
- pearson_baseline, k=100, n_facotrs=100 (0.005128205128205128)
- pearson, k=100, n_facotrs=100 (0.005128205128205128)
- cosine, k=100, n_facotrs=100 (0.007692307692307694)
- msd, k=100, n_facotrs=100 (0.007692307692307694)

In [287]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'k': [1, 100],
    'sim_options': {'name': ['pearson'], 'n_factors': [100, 200, 300]}
}

gs = GridSearchCV(KNNBaseline, param_grid, measures=['mae'], cv=5)

# 데이터에 대해 그리드 탐색 수행
reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als.

{'mae': {'k': 100,
  'sim_options': {'name': 'pearson', 'n_factors': 100, 'user_based': True}}}

In [177]:
sim_options = {'name': 'pearson',
               'user_based': False  # item-based similarity(사용자 기반 협업 필터)
               }
# 최근접 이웃의 수를 6으로 지정, 훈련할 때 사용되는 난수 발생 시드 지정
knnbaseline_i = KNNBaseline(k = 20, sim_options=sim_options, random_state = 42)
knnbaseline_i.fit(trainset)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [178]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(knnbaseline_i.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
# result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,real,predict
0,59879000,0,None,0.089332
1,59879000,1,None,0.091833
2,59879000,3,None,0.089181
3,59879000,5,None,0.089213
4,59879000,6,None,0.115409
...,...,...,...,...
1174350,67160000,4708,None,0.067741
1174351,67160000,4710,None,0.069055
1174352,67160000,4711,None,0.068609
1174353,67160000,4712,None,0.068272


In [180]:
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.089332
1,59879000,1,0.091833
2,59879000,3,0.089181
3,59879000,5,0.089213
4,59879000,6,0.115409
...,...,...,...
1174350,67160000,4708,0.067741
1174351,67160000,4710,0.069055
1174352,67160000,4711,0.068609
1174353,67160000,4712,0.068272


In [181]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2092,143,1168,1166,2537,296,8,1267,2288,...,1191,554,3838,335,784,618,1200,728,652,1243
1,59882000,143,1166,8,2537,2288,3873,2865,2513,3322,...,613,554,256,7,3838,335,333,2630,4064,652
2,59886000,3891,296,2537,2865,1166,2288,2057,4018,1267,...,1156,613,116,3314,2850,1277,4209,2608,2058,818
3,59892000,143,1166,296,2865,2288,738,2537,8,1329,...,554,1200,3887,2608,610,16,1243,2058,2336,553
4,59895000,738,747,1885,958,784,1156,2537,1166,296,...,780,432,613,2057,2608,538,2172,759,818,401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,67130000,143,2865,1074,313,2147,3933,2537,607,1277,...,490,3240,3951,1434,723,4537,1249,3148,842,259
441,67137000,143,2537,296,2865,2288,738,1166,1200,8,...,7,3576,652,2633,759,476,2630,256,364,2866
442,67140000,245,313,8,2984,563,4142,1440,607,3462,...,1858,2140,2783,3078,575,2959,3910,1073,1947,3224
443,67149000,245,335,538,1150,313,143,8,4209,2984,...,3579,3519,173,613,332,608,988,391,771,3177


In [182]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2092, 143, 1168, 1166, 2537, 296, 8, 1267, 22..."
1,59882000,"[143, 1166, 8, 2537, 2288, 3873, 2865, 2513, 3..."
2,59886000,"[3891, 296, 2537, 2865, 1166, 2288, 2057, 4018..."
3,59892000,"[143, 1166, 296, 2865, 2288, 738, 2537, 8, 132..."
4,59895000,"[738, 747, 1885, 958, 784, 1156, 2537, 1166, 2..."
...,...,...
440,67130000,"[143, 2865, 1074, 313, 2147, 3933, 2537, 607, ..."
441,67137000,"[143, 2537, 296, 2865, 2288, 738, 1166, 1200, ..."
442,67140000,"[245, 313, 8, 2984, 563, 4142, 1440, 607, 3462..."
443,67149000,"[245, 335, 538, 1150, 313, 143, 8, 4209, 2984,..."


In [183]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2092.0, 143.0, 1168.0, 1166.0, 2537.0, 296.0,..."
1,59882000,"[143.0, 1166.0, 8.0, 2537.0, 2288.0, 3873.0, 2..."
2,59886000,"[3891.0, 296.0, 2537.0, 2865.0, 1166.0, 2288.0..."
3,59892000,"[143.0, 1166.0, 296.0, 2865.0, 2288.0, 738.0, ..."
4,59895000,"[738.0, 747.0, 1885.0, 958.0, 784.0, 1156.0, 1..."
...,...,...
440,67130000,"[143.0, 2865.0, 1074.0, 313.0, 2147.0, 3933.0,..."
441,67137000,"[143.0, 2537.0, 296.0, 2865.0, 2288.0, 738.0, ..."
442,67140000,"[245.0, 313.0, 8.0, 2984.0, 563.0, 4142.0, 144..."
443,67149000,"[245.0, 335.0, 538.0, 1150.0, 313.0, 143.0, 8...."


In [184]:
testdata = pd.read_csv('../data/전처리data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [185]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.025595238095238098

### SVD
- SVD basic (0.007692307692307694)
- n_epochs= 95, lr_all= 0.005, reg_all= 0.06, random_state= 42 (0.012307692307692311)
- n_epochs= 295, lr_all= 0.05, reg_all= 0.01, random_state= 42 (0.014871794871794878)

#### 3개 이상
- n_epochs= 80, lr_all= 0.07, reg_all= 0.03, random_state= 42, n_factors=100 (0.0)
- n_epochs= 60, lr_all= 0.04, reg_all= 0.05, random_state= 42, n_factors=50 (0.0)
- n_epochs= 60, lr_all= 0.02, reg_all= 0.08, random_state= 42, n_factors=20(0.0)

#### 전체
- n_epochs= 50, lr_all= 0.02, reg_all= 0.08, random_state= 42, n_factors=20 (0.0)

In [291]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(30, 100), 'lr_all': [0.001, 0.03],
              'reg_all': [0.01, 0.1], 'random_state' : [42], 'n_factors': (1, 80)}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

{'mae': {'n_epochs': 30,
  'lr_all': 0.03,
  'reg_all': 0.01,
  'random_state': 42,
  'n_factors': 1}}

In [ ]:
svd = SVD(n_epochs= 295, lr_all= 0.05, reg_all= 0.01, random_state= 42)
svd.fit(trainset)

In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svd.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.211147
1,59879000,1,0.102272
2,59879000,3,0.164516
3,59879000,5,0.156392
4,59879000,6,0.180242
...,...,...,...
2409315,67164000,4708,0.558609
2409316,67164000,4710,0.619243
2409317,67164000,4711,0.517660
2409318,67164000,4712,0.366886


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,1863,2226,979,868,1243,4223,32,137,1772,...,245,882,3183,849,738,2092,3112,10,3426,976
1,59882000,3183,979,2226,137,882,868,1227,2307,32,...,3838,3196,726,9,1243,3705,3442,3568,139,991
2,59886000,979,137,754,3183,2226,1329,1243,3141,3705,...,3776,139,9,1956,1599,882,3906,414,249,555
3,59890000,2226,754,2716,137,3183,1329,3141,976,3705,...,4293,3126,824,448,1599,3906,1670,1013,10,152
4,59892000,2226,137,1958,868,754,32,3141,1599,3705,...,3183,849,728,1863,976,3838,360,908,10,1243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,868,3141,1865,1863,1772,3196,979,849,137,...,3838,991,2148,3705,139,1243,10,249,1958,112
895,67154000,476,32,754,1401,849,316,1863,1958,1107,...,3183,137,1705,1191,976,4121,1187,1167,3781,1025
896,67160000,1227,32,3141,2148,1863,137,315,9,991,...,245,3838,4223,849,3442,1599,1958,1243,1635,908
897,67161000,4612,3141,4040,458,4223,1191,868,2226,9,...,3678,2993,991,2148,1906,3598,4091,1738,3570,1575


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[1863, 2226, 979, 868, 1243, 4223, 32, 137, 17..."
1,59882000,"[3183, 979, 2226, 137, 882, 868, 1227, 2307, 3..."
2,59886000,"[979, 137, 754, 3183, 2226, 1329, 1243, 3141, ..."
3,59890000,"[2226, 754, 2716, 137, 3183, 1329, 3141, 976, ..."
4,59892000,"[2226, 137, 1958, 868, 754, 32, 3141, 1599, 37..."
...,...,...
894,67149000,"[868, 3141, 1865, 1863, 1772, 3196, 979, 849, ..."
895,67154000,"[476, 32, 754, 1401, 849, 316, 1863, 1958, 110..."
896,67160000,"[1227, 32, 3141, 2148, 1863, 137, 315, 9, 991,..."
897,67161000,"[4612, 3141, 4040, 458, 4223, 1191, 868, 2226,..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[1863.0, 2226.0, 979.0, 868.0, 1243.0, 4223.0,..."
1,59882000,"[3183.0, 979.0, 2226.0, 137.0, 882.0, 868.0, 1..."
2,59886000,"[979.0, 137.0, 754.0, 3183.0, 2226.0, 1329.0, ..."
3,59890000,"[2226.0, 754.0, 137.0, 3183.0, 1329.0, 3141.0,..."
4,59892000,"[2226.0, 137.0, 1958.0, 868.0, 754.0, 32.0, 31..."
...,...,...
894,67149000,"[868.0, 3141.0, 1865.0, 1863.0, 1772.0, 3196.0..."
895,67154000,"[476.0, 32.0, 754.0, 1401.0, 849.0, 316.0, 186..."
896,67160000,"[1227.0, 32.0, 3141.0, 2148.0, 1863.0, 137.0, ..."
897,67161000,"[4612.0, 3141.0, 4040.0, 458.0, 4223.0, 1191.0..."


In [ ]:
testdata = pd.read_csv('/content/drive/MyDrive/LG Hellovision VOD 추천 프로젝트/모델/content_surprise/csv/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

,subsr,vod_id
0,59900000,"[1278, 1885]"
1,59930000,[2098]
2,59933000,[296]
3,60050000,"[200, 2415, 2869, 2546, 213, 2291, 2610, 4880]"
4,60067000,"[1785, 887, 1347, 4773, 4782, 4759, 992, 3806,..."
...,...,...
215,67133000,[296]
216,67164000,"[4956, 1434, 4995, 2058]"
217,67170000,"[3464, 948, 3900, 675]"
218,67202000,[1028]


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict_1)

0.014871794871794878

### SVDpp (0.0005154639175257732)

#### 3개 이상
- n_epochs= 145, lr_all= 0.01, reg_all= 0.09, random_state= 42 (0.00625)
- n_epochs= 130, lr_all= 0.03, reg_all= 0.07, random_state= 42 (0.00625)
- n_epochs= 100, lr_all= 0.08, reg_all= 0.03, random_state= 42 (0.00625)

#### 전체

In [269]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': np.arange(100, 180, 5), 'lr_all': [0.005, 0.08],
              'reg_all': [0.03, 0.2], 'random_state' : [42]}
              # lr_all – The learning rate for all parameters. Default is 0.005.
              # reg_all – The regularization term for all parameters. Default is 0.02.

gs = GridSearchCV(SVD, param_grid, measures=['mae'], cv=5)

reader = Reader(rating_scale=(0,1))
train_set = Dataset.load_from_df(con_score[['subsr', 'vod_id', 'click_score']], reader)

gs.fit(train_set)
gs.best_params

{'mae': {'n_epochs': 100, 'lr_all': 0.08, 'reg_all': 0.03, 'random_state': 42}}

In [ ]:
svdpp = SVDpp(n_epochs= 95, lr_all= 0.005, reg_all= 0.06, random_state= 42)
svdpp.fit(trainset)

In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(svdpp.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.273517
1,59879000,1,0.099659
2,59879000,3,0.228638
3,59879000,5,0.190729
4,59879000,6,0.200687
...,...,...,...
2409315,67164000,4708,0.261200
2409316,67164000,4710,0.381770
2409317,67164000,4711,0.231248
2409318,67164000,4712,0.102031


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,3838,1243,784,1863,979,2537,789,1191,498,...,112,2838,1921,4223,2288,3954,510,1214,1599,2226
1,59882000,3183,296,789,2537,2307,2288,3285,1243,112,...,1027,1227,240,882,2336,293,1599,868,1617,818
2,59886000,979,2576,137,112,1329,3183,308,555,1958,...,144,3891,3141,2302,2556,1233,1243,2667,734,388
3,59890000,2716,784,137,1329,3183,754,3779,3705,3838,...,296,538,1749,1877,448,4146,498,1050,868,2221
4,59892000,2867,1958,2537,868,137,1233,144,728,2288,...,2865,10,3029,2560,240,538,296,4672,445,3705
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,1191,3141,868,1863,3196,1243,296,2537,1166,...,2290,581,2288,3891,1772,2865,3272,112,1709,1865
895,67154000,476,32,847,1401,4121,240,3838,1191,1877,...,3196,2875,549,2288,112,756,509,1107,538,784
896,67160000,4114,1214,112,2718,4446,821,32,2343,1227,...,316,2537,2109,2148,245,3768,610,296,2057,1786
897,67161000,1191,868,3824,458,4612,4101,4223,2226,1575,...,3705,4121,2949,433,3196,1943,818,320,2024,4469


In [ ]:
svdpp_predict = result_1.copy()
svdpp_predict.index = user_id
svdpp_predict = svdpp_predict.apply(lambda x : x[1:].tolist(), axis = 1)
svdpp_predict = svdpp_predict.reset_index()
svdpp_predict.columns = ['subsr', 'vod_id']
svdpp_predict

,subsr,vod_id
0,59879000,"[3838, 1243, 784, 1863, 979, 2537, 789, 1191, ..."
1,59882000,"[3183, 296, 789, 2537, 2307, 2288, 3285, 1243,..."
2,59886000,"[979, 2576, 137, 112, 1329, 3183, 308, 555, 19..."
3,59890000,"[2716, 784, 137, 1329, 3183, 754, 3779, 3705, ..."
4,59892000,"[2867, 1958, 2537, 868, 137, 1233, 144, 728, 2..."
...,...,...
894,67149000,"[1191, 3141, 868, 1863, 3196, 1243, 296, 2537,..."
895,67154000,"[476, 32, 847, 1401, 4121, 240, 3838, 1191, 18..."
896,67160000,"[4114, 1214, 112, 2718, 4446, 821, 32, 2343, 1..."
897,67161000,"[1191, 868, 3824, 458, 4612, 4101, 4223, 2226,..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
# vod_list = pd.read_csv('../data/vod_list_add10.csv', index_col=0)
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

svdpp_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    svdpp_predict_1 = pd.concat([svdpp_predict_1, pd.DataFrame(li).T], axis= 0)

svdpp_predict_1.index = user_id
svdpp_predict_1 = svdpp_predict_1.apply(lambda x : x.tolist(), axis = 1)
svdpp_predict_1 = svdpp_predict_1.reset_index()
svdpp_predict_1.columns = ['subsr', 'vod_id']
svdpp_predict_1

,subsr,vod_id
0,59879000,"[3838.0, 1243.0, 784.0, 1863.0, 979.0, 2537.0,..."
1,59882000,"[3183.0, 296.0, 789.0, 2537.0, 2307.0, 2288.0,..."
2,59886000,"[979.0, 2576.0, 137.0, 112.0, 1329.0, 3183.0, ..."
3,59890000,"[784.0, 137.0, 1329.0, 3183.0, 754.0, 3779.0, ..."
4,59892000,"[2867.0, 1958.0, 868.0, 137.0, 1233.0, 144.0, ..."
...,...,...
894,67149000,"[1191.0, 3141.0, 868.0, 1863.0, 3196.0, 1243.0..."
895,67154000,"[476.0, 32.0, 847.0, 1401.0, 4121.0, 240.0, 38..."
896,67160000,"[4114.0, 1214.0, 112.0, 2718.0, 4446.0, 821.0,..."
897,67161000,"[1191.0, 868.0, 458.0, 4612.0, 4101.0, 4223.0,..."


In [ ]:
testdata = pd.read_csv('../data/watched_vod_10.csv', index_col=0)
testdata = testdata.groupby('subsr')['vod_id'].unique().reset_index()
testdata

In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, svdpp_predict_1)

0.012307692307692311

### BaselineOnly (0.012307692307692311)

In [ ]:
bsl_options = {'method' : 'als', 'n_epochs' : 60}
baseline = BaselineOnly(bsl_options=bsl_options)
baseline.fit(trainset)

Estimating biases using als...


In [ ]:
user_id = sorted(con_score.subsr.unique())
vod_id = sorted(con_score.vod_id.unique())

result = []
for user in user_id:
    for vod in vod_id:
        result.append(baseline.predict(user, vod)[0:4])

result = pd.DataFrame(result, columns = ['subsr', 'vod_id', 'real', 'predict'])
result = result[['subsr', 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.129002
1,59879000,1,0.129262
2,59879000,3,0.184380
3,59879000,5,0.127621
4,59879000,6,0.151202
...,...,...,...
2409315,67164000,4708,0.145030
2409316,67164000,4710,0.145343
2409317,67164000,4711,0.145322
2409318,67164000,4712,0.145528


In [ ]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
1,59882000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
2,59886000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
3,59890000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
4,59892000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
895,67154000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
896,67160000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256
897,67161000,2537,2288,2865,2057,296,2608,1166,538,143,...,401,2866,847,8,112,2058,4018,3838,336,256


In [ ]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
1,59882000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
2,59886000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
3,59890000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
4,59892000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
...,...,...
894,67149000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
895,67154000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
896,67160000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."
897,67161000,"[2537, 2288, 2865, 2057, 296, 2608, 1166, 538,..."


In [ ]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[3838.0, 1243.0, 784.0, 1863.0, 979.0, 2537.0,..."
1,59882000,"[3183.0, 296.0, 789.0, 2537.0, 2307.0, 2288.0,..."
2,59886000,"[979.0, 2576.0, 137.0, 112.0, 1329.0, 3183.0, ..."
3,59890000,"[784.0, 137.0, 1329.0, 3183.0, 754.0, 3779.0, ..."
4,59892000,"[2867.0, 1958.0, 868.0, 137.0, 1233.0, 144.0, ..."
...,...,...
894,67149000,"[1191.0, 3141.0, 868.0, 1863.0, 3196.0, 1243.0..."
895,67154000,"[476.0, 32.0, 847.0, 1401.0, 4121.0, 240.0, 38..."
896,67160000,"[4114.0, 1214.0, 112.0, 2718.0, 4446.0, 821.0,..."
897,67161000,"[1191.0, 868.0, 458.0, 4612.0, 4101.0, 4223.0,..."


In [ ]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict, K = 10)

0.012307692307692311

### 앙상블 모델 (0.03230769230769227)

In [187]:
knnbaseline_u_predict = []
knnbaseline_i_predict = []
for user in user_id:
    for vod in vod_id:
        knnbaseline_u_predict.append(knnbaseline_u.predict(user, vod)[0:4])
        knnbaseline_i_predict.append(knnbaseline_u.predict(user, vod)[0:4])

knnbaseline_u_predict = pd.DataFrame(knnbaseline_u_predict, columns = ['subsr', 'vod_id', 'real', 'predict'])
knnbaseline_u_predict = knnbaseline_u_predict[['subsr', 'vod_id', 'predict']]

knnbaseline_i_predict = pd.DataFrame(knnbaseline_i_predict, columns = ['subsr', 'vod_id', 'real', 'predict'])
knnbaseline_i_predict = knnbaseline_i_predict[['subsr', 'vod_id', 'predict']]

In [188]:
result = knnbaseline_i_predict.merge(knnbaseline_i_predict, how = 'left', on = ['subsr', 'vod_id'])
result['predict'] = (result['predict_x'] + result['predict_y']) / 2
result = result[['subsr' , 'vod_id', 'predict']]
result

,subsr,vod_id,predict
0,59879000,0,0.089332
1,59879000,1,0.091833
2,59879000,3,0.089181
3,59879000,5,0.089213
4,59879000,6,0.115409
...,...,...,...
1174350,67160000,4708,0.057524
1174351,67160000,4710,0.069055
1174352,67160000,4711,0.068609
1174353,67160000,4712,0.068272


In [189]:
result_1 = []
for user in user_id:
    result_1.append(result[result['subsr'] == user].sort_values(by = 'predict', ascending=False).vod_id[:30].tolist())

result_1 = pd.DataFrame(result_1)
result_1 = pd.concat([pd.DataFrame(user_id, columns = ['subsr']), result_1], axis = 1)
result_1

,subsr,0,1,2,3,4,5,6,7,8,...,20,21,22,23,24,25,26,27,28,29
0,59879000,2092,143,1166,2537,296,8,2288,2865,1329,...,1200,652,1243,7,256,553,2336,518,780,432
1,59882000,143,336,2226,3026,1233,610,552,991,726,...,245,1191,4442,4176,3838,335,364,531,4167,553
2,59886000,3891,818,1398,143,1166,2537,296,8,2288,...,335,784,1200,652,1243,7,256,553,2336,518
3,59892000,738,2226,1200,976,849,1233,249,709,991,...,2232,842,784,2058,15,297,2537,1166,296,862
4,59895000,784,296,2057,2537,249,143,1166,8,2633,...,3838,335,1200,1243,7,256,553,518,780,123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,67130000,335,143,296,2172,1166,8,1156,1329,738,...,333,648,835,3912,3838,531,784,1200,1243,618
441,67137000,738,910,2057,2866,4018,1166,8,1329,1267,...,1243,7,2865,256,553,2336,518,780,432,123
442,67140000,3838,139,2984,2057,738,748,8,570,2865,...,1191,2028,4185,3912,360,1267,143,2172,452,1329
443,67149000,1166,2057,2537,143,652,296,8,888,1758,...,4018,759,1191,554,3838,335,784,531,2336,1200


In [190]:
vod_predict = result_1.copy()
vod_predict.index = user_id
vod_predict = vod_predict.apply(lambda x : x[1:].tolist(), axis = 1)
vod_predict = vod_predict.reset_index()
vod_predict.columns = ['subsr', 'vod_id']
vod_predict

,subsr,vod_id
0,59879000,"[2092, 143, 1166, 2537, 296, 8, 2288, 2865, 13..."
1,59882000,"[143, 336, 2226, 3026, 1233, 610, 552, 991, 72..."
2,59886000,"[3891, 818, 1398, 143, 1166, 2537, 296, 8, 228..."
3,59892000,"[738, 2226, 1200, 976, 849, 1233, 249, 709, 99..."
4,59895000,"[784, 296, 2057, 2537, 249, 143, 1166, 8, 2633..."
...,...,...
440,67130000,"[335, 143, 296, 2172, 1166, 8, 1156, 1329, 738..."
441,67137000,"[738, 910, 2057, 2866, 4018, 1166, 8, 1329, 12..."
442,67140000,"[3838, 139, 2984, 2057, 738, 748, 8, 570, 2865..."
443,67149000,"[1166, 2057, 2537, 143, 652, 296, 8, 888, 1758..."


In [191]:
# 추천 VOD가 영화인 경우, 본 적이 있다면 추천안함
TV_kids = vod_list[(vod_list['Category'] == 'TV프로그램') | (vod_list['Category'] == '키즈')].vod_id.unique().tolist()
movie = vod_list[vod_list['Category'] == '영화'].vod_id.unique().tolist()

vod_predict_1 = pd.DataFrame()
for user in user_id:
    li = []
    watched_list = con_score[con_score.subsr == user].vod_id.tolist()
    rec_list = vod_predict[vod_predict.subsr == user].vod_id.tolist()[0]
    for x in rec_list:
        if x not in watched_list:
            li.append(x)
        elif x in watched_list and x in TV_kids:
            li.append(x)
        elif x in watched_list and x in movie:
            continue

    vod_predict_1 = pd.concat([vod_predict_1, pd.DataFrame(li).T], axis= 0)

vod_predict_1.index = user_id
vod_predict_1 = vod_predict_1.apply(lambda x : x.tolist(), axis = 1)
vod_predict_1 = vod_predict_1.reset_index()
vod_predict_1.columns = ['subsr', 'vod_id']
vod_predict_1

,subsr,vod_id
0,59879000,"[2092.0, 143.0, 1166.0, 2537.0, 296.0, 8.0, 22..."
1,59882000,"[143.0, 336.0, 2226.0, 3026.0, 1233.0, 610.0, ..."
2,59886000,"[3891.0, 818.0, 143.0, 1166.0, 2537.0, 296.0, ..."
3,59892000,"[738.0, 2226.0, 1200.0, 976.0, 849.0, 1233.0, ..."
4,59895000,"[784.0, 296.0, 2057.0, 249.0, 143.0, 1166.0, 8..."
...,...,...
440,67130000,"[335.0, 143.0, 296.0, 1166.0, 8.0, 1156.0, 132..."
441,67137000,"[738.0, 910.0, 4018.0, 1166.0, 8.0, 1329.0, 12..."
442,67140000,"[3838.0, 139.0, 2984.0, 738.0, 748.0, 8.0, 570..."
443,67149000,"[1166.0, 2537.0, 143.0, 652.0, 296.0, 8.0, 888..."


In [192]:
# precision@k
def precision_k(testdata, recommended_data, K = 10):
    merge_df = pd.merge(testdata, recommended_data, on='subsr', how = 'left', suffixes=('_actual', '_rec'))
    merge_df = merge_df.dropna()
    precision_value = 0
    for i in range(merge_df.shape[0]):
        temp = len(set(merge_df.iloc[i,1]).intersection(set(merge_df.iloc[i, 2][:K]))) / K
        precision_value += temp

    return precision_value / merge_df.shape[0]

precision_k(testdata, vod_predict)

0.03154761904761904

In [65]:
content_ensemble = result_1.iloc[:, :11]
content_ensemble

,subsr,0,1,2,3,4,5,6,7,8,9
0,59879000,2537,2288,2865,2057,296,2092,2608,1166,538,143
1,59882000,1233,296,3026,538,143,888,2867,1877,581,240
2,59886000,3891,2537,2288,2865,2057,296,2608,1166,538,143
3,59890000,2716,2537,2288,2865,2057,296,2608,1166,538,143
4,59892000,2288,2865,2057,2608,538,143,888,2867,2633,2630
...,...,...,...,...,...,...,...,...,...,...,...
894,67149000,297,2226,610,991,538,143,726,1877,1233,581
895,67154000,476,2537,2288,2865,2057,296,2608,1166,538,143
896,67160000,2537,2288,2865,296,2608,1166,538,888,2867,2633
897,67161000,3824,2537,2288,2865,2057,296,2608,1166,538,143


In [67]:
# content_ensemble.to_csv('../data/전처리data/content_ensemble.csv', encoding='cp949', index = False)